# Predicting US Immigration Quota

## Current variables: US population, Global population, US GDP, Global GDP, Foreign Policy indicator.

## Potential variables: US tuition rates, Global tuition rates, occupation, and salary

## Imports

In [1]:
from __future__ import print_function, division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot
import pylab
import pickle
import patsy
%matplotlib inline
%config InlineBackend.figure_format='svg'
import scipy
import statsmodels
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.cross_validation import KFold

/Users/cyrusrustomji/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/Users/cyrusrustomji/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error as MSE
from sklearn.linear_model import Lasso

In [3]:
import datetime as dt

## Webscrape

# Create lists and put into a pandas dataframe

> Find a way to keep 1971

# New Citizens by Year

> Will look into outliers from 1990 - 1993, thinking gulf war, civil wars and rebellions in Africa will be the issue

## Pickle files

In [4]:
# write the table to a picklefile, then read the pickle

# combine_table = combine_table.to_pickle('new_citizens.pkl')
combine_table = pd.read_pickle('new_citizens.pkl')
# df = df.to_pickle('first_table.pkl')
df = pd.read_pickle('first_table.pkl')

FileNotFoundError: [Errno 2] No such file or directory: 'new_citizens.pkl'

# Create an OLS and do an EDA

In [ ]:
combine_table.head()

In [ ]:
# why they spiked in 1990 - 1993, gulf war

x = df[:50].new_citizens
y = df[:50].year
plt.title('New Citizens by Year')
plt.xlabel("Year")
plt.ylabel("Population")
matplotlib.pyplot.scatter(y,x)
p = matplotlib.pyplot.show();
plt.savefig('Pop.png', dpi=300)

In [ ]:
# this shows a linear correlation with the US and Global Populations

sns.pairplot(combine_table, vars=['US_Population', 'Global_Population'], kind='reg');

In [ ]:
# shows as how GDP rises, foreign policy goes to a more lax approach

# sns.lmplot(y='US_GDP', x='Foreign_Policy', data = combine_table);
sns.lmplot(y='Foreign_Policy', x='US_GDP', data = combine_table);

In [ ]:
# preform an ANOVA test on current data

new = ols('new_citizens ~ US_GDP + US_Population + World_GDP + Global_Population + Foreign_Policy', data=combine_table).fit()
table = sm.stats.anova_lm(new, typ=2) # Type 2 ANOVA DataFrame

print(table)

In [ ]:
combine_table.corr()

In [ ]:
'''all variables are heavily correlated. Foreign Policy is right in the middle because there is an
even 50% split for each as shown in the line below'''
# global population has the highest correlation with new citizens
# need to confirm the p-value, t-statistic, and more if this is the strongest variable or not

sns.heatmap(combine_table.corr(), cmap='seismic',vmin=-1,vmax=1, annot=True);
plt.savefig('correlation.png', bbox_inches='tight', dpi=300)

In [ ]:
# use conditional probability on foreign policy
# currently goes back until 1951
# Based it off of the president because when a president was elected.
# Congress's party fell in line with the party of
# the president except when Nixon, Ford, HW were president.

# Plot all of the variable-to-variable relations as scatterplots
# histograms are within themselves because it is the distribution against itself
# counts the bins the y axis against itself in the x

rating_fp = combine_table.groupby('Foreign_Policy').size().div(len(combine_table))
rating_fp

In [ ]:
sns.pairplot(combine_table, size =1.5, aspect=1);

# Regression

In [ ]:
y, X = patsy.dmatrices('new_citizens ~ US_GDP + World_GDP + US_Population + Global_Population + Foreign_Policy', data=combine_table, return_type="dataframe")

# Create model
model1 = sm.OLS(y,X)
# This model fits the whole model to my as my training set
fit1 = model1.fit()
# Print summary statistics of the model's performance
fit1.summary()

> Since the R^2 dropped and the adjusted R^2 only increased by 0.002, I will include both GDPs even though their p-values are high

In [ ]:
# use the first fit because the r^2 dropped by taking out the GDPs, even though their p-values are
# high

fit1.summary()

## Accuracy Score

In [ ]:
lr1 = LinearRegression()

X = combine_table.iloc[:,1:6]
# All variables in the data set will be the response variables
y = combine_table.iloc[:,0]
# Fit the model to the full dataset
lr1.fit(X,y)
# Print out the R^2 for the model against the full dataset
lr1.score(X,y)

In [ ]:
X.head()

In [ ]:
# show the intercept of the linear regression
print(lr1.intercept_)
# show the coefficients of each section in the linear regression
lr1.coef_

# Log New Citizens (y value)

In [ ]:
combine_table.new_citizens.hist();

In [ ]:
combine_table['new_citizens_log']=np.log(combine_table.new_citizens)

In [ ]:
combine_table

In [ ]:
sns.pairplot(combine_table, size =1.5, aspect=1);

In [ ]:
combine_table.head()

In [ ]:
sns.pairplot(combine_table);

In [ ]:
y, X = patsy.dmatrices('new_citizens_log ~ US_GDP + World_GDP + US_Population + Global_Population + Foreign_Policy', data=combine_table, return_type="dataframe")

log_model = sm.OLS(y,X)
log_fit = log_model.fit()
log_fit.summary()

> Remove both GDPs b/c high p-values, then possibly foreign policy

> Keep both GDPs b/c R^2 lower by 0.001, adjusted R^2 only went up by 0.003

In [ ]:
log_fit.summary() # first log

In [ ]:
from sklearn.cross_validation import cross_val_score
reg = LinearRegression()
scores = cross_val_score(reg, X, y, cv=10, scoring='mean_squared_error')

# scores output is negative, a sklearn quirk bc mse is used to min. optimization func.
print(-scores)

In [ ]:
np.mean(-scores)

### Accuracy Score of model without logging using train/test split

In [ ]:
linear = LinearRegression()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
linear.fit(X_train,y_train)
linear.score(X_test,y_test)

In [ ]:
sns.heatmap(combine_table.corr(), cmap='seismic',vmin=-1,vmax=1, annot=True);

In [ ]:
combine_table.head()

In [ ]:
lr5 = LinearRegression()

X = combine_table.iloc[:,1:6]
# All variables in the data set will be the response variables
y = combine_table.iloc[:,-1]
# Fit the model to the full dataset
lr1.fit(X,y)
# Print out the R^2 for the model against the full dataset
lr1.score(X,y)

In [ ]:
combine_table.head()

In [ ]:
# combine_table_log = combine_table_log.to_pickle('new_citizens_log.pkl')
combine_table_log = pd.read_pickle('new_citizens_log.pkl')

# Regularization

In [ ]:
combine_table_log.head()

In [ ]:
X = combine_table_log.drop(['new_citizens_log','Year'],axis=1)
y = combine_table_log['new_citizens_log']
X.shape
kf = KFold(n_splits=5,shuffle=True)

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
# Elastic Net

from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
alpha = 1e-5
enet = ElasticNet(alpha=alpha)
y_pred_enet = enet.fit(X, y)
y_pred_enet.score(X,y)

> Do a Polynomial Fit

In [ ]:
from scipy import stats
from sklearn import preprocessing

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(combine_table)
combine_normalized = pd.DataFrame(np_scaled)
combine_normalized.head()

In [ ]:
combine_table.head()

> Do Regularization

In [ ]:
# do regularization on all variables (use lasso for GDPs because both coeeffs are high)
    # lasso brings coeffs to 0
# look at how conditional, continuous, and intro to prob could affect the data

In [ ]:
# what to do for foreign policy

# have training sets and test sets
# have multiple training and test sets
# we want cross validation (test error) is the lowest